In [1]:
import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import LearningRateScheduler
import math
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

> In the paper, CNN512 with dropout achieves the highest accuracy 0.841

## *The hyperparameter configuration of CNNs.*

Configuration	Values

Optimizer	SGD

Momentum	0.9

Max Learning rate	1×10−1 in custom CNNs

1×10−2 in EfficientNetB0

Base Learning rate	1×10−4

Mode	triangular

Class weight	auto

Dropout	0.5

Augmentation	20 times (I didn't include augmentation in the following code since we decide not to do it)

## *CNN512* 

Layer	Operator	Layer Details

Input Layer	Zero Padding layer	Padding (2,2)

Layer 1	2D CONV layer	Kernel number = 32, kernel size = 3

Layer 2	Batch Normalization layer	-

Layer 3	Relu layer	-

Layer 4	Max Pooling layer	Pooling size (2,2)

Layer 5	2D CONV layer	Kernel number = 64, kernel size = 3

Layer 6	Batch Normalization layer	-

Layer 7	Relu layer	-

Layer 8	Max Pooling layer	Pooling size (2,2)

Layer 9	2D CONV layer	Kernel number = 96, kernel size = 3

Layer 10	Batch Normalization layer	-

Layer 11	Relu layer	-

Layer 12	Max Pooling layer	Pooling size (2,2)

Layer 13	2D CONV layer	Kernel number = 96, kernel size = 3

Layer 14	Batch Normalization layer	-

Layer 15	Relu layer	-

Layer 16	Max Pooling layer	Pooling size (2,2)

Layer 17	2D CONV layer	Kernel number = 128, kernel size = 3

Layer 18	Batch Normalization layer	-

Layer 19	Relu layer	-

Layer 20	Max Pooling layer	Pooling size (2,2)

Layer 21	2D CONV layer	Kernel number = 200, kernel size = 3

Layer 22	Batch Normalization layer	-

Layer 23	Relu layer	-

Layer 24	Max Pooling layer	Pooling size (2,2)

Layer 25	Flatten layer	-

Layer 26	FC layer	Neurons number = 1000

Layer 27	Batch Normalization layer	-

Layer 28	Relu layer	-

Layer 29	FC layer	Neurons number = 500

Layer 30	Batch Normalization layer	-

Layer 31	Relu layer	-

Layer 32	FC layer	With SoftMax activation


In [4]:
input_shape = (512, 512, 3)  # placeholder
num_classes = 5
epochs = 10  # placeholder

In [ ]:
x_train, y_train, x_test, y_test = load_data()  # placeholder
x_train = x_train.astype("float32") / 225  # normalize
x_test = x_test.astype("float32") / 225  # normalize

CNN512 Without dropout

In [6]:
cnn512_nodropout = models.Sequential(
    [
        # Input Layer	Zero Padding layer	Padding (2,2)
        layers.ZeroPadding2D(padding=(2, 2), input_shape=input_shape),
        # Layer 1	2D CONV layer	Kernel number = 32, kernel size = 3
        layers.Conv2D(32, kernel_size=(3, 3), padding="same"),
        # Layer 2	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 3	Relu layer	-
        layers.Activation("relu"),
        # Layer 4	Max Pooling layer	Pooling size (2,2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 5	2D CONV layer	Kernel number = 64, kernel size = 3
        layers.Conv2D(64, kernel_size=(3, 3), padding="same"),
        # Layer 6	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 7	Relu layer	-
        layers.Activation("relu"),
        # Layer 8	Max Pooling layer	Pooling size (2, 2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 9	2D CONV layer	Kernel number = 96, kernel size = 3
        layers.Conv2D(96, kernel_size=(3, 3), padding="same"),
        # Layer 10	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 11	Relu layer	-
        layers.Activation("relu"),
        # Layer 12	Max Pooling layer	Pooling size (2, 2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 13	2D CONV layer	Kernel number = 96, kernel size = 3
        layers.Conv2D(96, kernel_size=(3, 3), padding="same"),
        # Layer 14	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 15	Relu layer	-
        layers.Activation("relu"),
        # Layer 16	Max Pooling layer	Pooling size (2, 2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 17	2D CONV layer	Kernel number = 128, kernel size = 3
        layers.Conv2D(128, kernel_size=(3, 3), padding="same"),
        # Layer 18	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 19	Relu layer	-
        layers.Activation("relu"),
        # Layer 20	Max Pooling layer	Pooling size (2, 2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 21	2D CONV layer	Kernel number = 200, kernel size = 3
        layers.Conv2D(200, kernel_size=(3, 3), padding="same"),
        # Layer 22	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 23	Relu layer	-
        layers.Activation("relu"),
        # Layer 24	Max Pooling layer	Pooling size (2, 2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 25	Flatten layer	-
        layers.Flatten(),
        # Layer 26	FC layer	Neurons number = 1000
        layers.Dense(1000),
        # Layer 27	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 28	Relu layer	-
        layers.Activation("relu"),
        # layers.Dropout(0.5),
        # Layer 29	FC layer	Neurons number = 500
        layers.Dense(500),
        # Layer 30	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 31	Relu layer	-
        layers.Activation("relu"),
        # layers.Dropout(0.5),
        # Layer 32	FC layer with Softmax activatio
        layers.Dense(num_classes, activation="softmax"),
    ]
)

/Users/tinayiluo/miniconda3/lib/python3.11/site-packages/keras/src/layers/reshaping/zero_padding2d.py:72: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [7]:
cnn512_nodropout.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ zero_padding2d (ZeroPadding2D)  │ (None, 516, 516, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 516, 516, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 516, 516, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 516, 516, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 258, 258, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 258, 258, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 258, 258, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 258, 258, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 129, 129, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 129, 129, 96)   │        55,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 129, 129, 96)   │           384 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 129, 129, 96)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 64, 64, 96)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 64, 64, 96)     │        83,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 64, 64, 96)     │           384 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ (None, 64, 64, 96)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 32, 32, 96)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 32, 32, 128)    │       110,720 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 32, 32, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_4 (Activation)       │ (None, 32, 32, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 16, 16, 200)    │       230,600 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 13,811,613 (52.69 MB)

 Trainable params: 13,807,381 (52.67 MB)

 Non-trainable params: 4,232 (16.53 KB)

In [ ]:
# base learning rate for custom CNNs
base_learning_rate = 1e-4
# maximum learning rate for custom CNNs
max_learning_rate = 1e-1

# Create an instance of SGD optimizer with initial learning rate
optimizer = SGD(learning_rate=base_learning_rate, momentum=0.9)

cnn512_nodropout.compile(
    optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

# create class weight
classes = np.unique(y_train)
class_weights = compute_class_weight("balanced", classes=classes, y=y_train)
class_weight_dict = dict(zip(classes, class_weights))


# create triangular schedule
def triangular_schedule(epoch):
    """Triangular learning rate scheduler."""
    cycle_length = 10  # Define the length of a cycle
    cycle = math.floor(1 + epoch / (2 * cycle_length))
    x = abs(epoch / cycle_length - 2 * cycle + 1)
    lr = base_learning_rate + (max_learning_rate - base_learning_rate) * max(0, (1 - x))
    return lr


# When fitting the model, include the learning rate scheduler callback
lr_scheduler = LearningRateScheduler(triangular_schedule)

# fitting the model
cnn512_nodropout.fit(
    x_train,
    y_train,
    epochs=epochs,
    class_weight=class_weight_dict,
    callbacks=[lr_scheduler],
)

# evaluating the model
cnn512_nodropout.evaluate(x_test, y_test)

CNN512 With Dropout

In [8]:
cnn512_dropout = models.Sequential(
    [
        # Input Layer	Zero Padding layer	Padding (2,2)
        layers.ZeroPadding2D(padding=(2, 2), input_shape=input_shape),
        # Layer 1	2D CONV layer	Kernel number = 32, kernel size = 3
        layers.Conv2D(32, kernel_size=(3, 3), padding="same"),
        # Layer 2	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 3	Relu layer	-
        layers.Activation("relu"),
        # Layer 4	Max Pooling layer	Pooling size (2,2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 5	2D CONV layer	Kernel number = 64, kernel size = 3
        layers.Conv2D(64, kernel_size=(3, 3), padding="same"),
        # Layer 6	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 7	Relu layer	-
        layers.Activation("relu"),
        # Layer 8	Max Pooling layer	Pooling size (2, 2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 9	2D CONV layer	Kernel number = 96, kernel size = 3
        layers.Conv2D(96, kernel_size=(3, 3), padding="same"),
        # Layer 10	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 11	Relu layer	-
        layers.Activation("relu"),
        # Layer 12	Max Pooling layer	Pooling size (2, 2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 13	2D CONV layer	Kernel number = 96, kernel size = 3
        layers.Conv2D(96, kernel_size=(3, 3), padding="same"),
        # Layer 14	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 15	Relu layer	-
        layers.Activation("relu"),
        # Layer 16	Max Pooling layer	Pooling size (2, 2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 17	2D CONV layer	Kernel number = 128, kernel size = 3
        layers.Conv2D(128, kernel_size=(3, 3), padding="same"),
        # Layer 18	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 19	Relu layer	-
        layers.Activation("relu"),
        # Layer 20	Max Pooling layer	Pooling size (2, 2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 21	2D CONV layer	Kernel number = 200, kernel size = 3
        layers.Conv2D(200, kernel_size=(3, 3), padding="same"),
        # Layer 22	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 23	Relu layer	-
        layers.Activation("relu"),
        # Layer 24	Max Pooling layer	Pooling size (2, 2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 25	Flatten layer	-
        layers.Flatten(),
        # Layer 26	FC layer	Neurons number = 1000
        layers.Dense(1000),
        # Layer 27	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 28	Relu layer	-
        layers.Activation("relu"),
        layers.Dropout(0.5),
        # Layer 29	FC layer	Neurons number = 500
        layers.Dense(500),
        # Layer 30	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 31	Relu layer	-
        layers.Activation("relu"),
        layers.Dropout(0.5),
        # Layer 32	FC layer with Softmax activatio
        layers.Dense(num_classes, activation="softmax"),
    ]
)

In [9]:
cnn512_dropout.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ zero_padding2d_1                │ (None, 516, 516, 3)    │             0 │
│ (ZeroPadding2D)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 516, 516, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 516, 516, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_8 (Activation)       │ (None, 516, 516, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 258, 258, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 258, 258, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_9           │ (None, 258, 258, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_9 (Activation)       │ (None, 258, 258, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 129, 129, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 129, 129, 96)   │        55,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_10          │ (None, 129, 129, 96)   │           384 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_10 (Activation)      │ (None, 129, 129, 96)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 64, 64, 96)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 64, 64, 96)     │        83,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_11          │ (None, 64, 64, 96)     │           384 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_11 (Activation)      │ (None, 64, 64, 96)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 32, 32, 96)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 32, 32, 128)    │       110,720 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_12          │ (None, 32, 32, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_12 (Activation)      │ (None, 32, 32, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 16, 16, 200)    │       230,60

 Total params: 13,811,613 (52.69 MB)

 Trainable params: 13,807,381 (52.67 MB)

 Non-trainable params: 4,232 (16.53 KB)

In [ ]:
# base learning rate for custom CNNs
base_learning_rate = 1e-4
# maximum learning rate for custom CNNs
max_learning_rate = 1e-1

# Create an instance of SGD optimizer with initial learning rate
optimizer = SGD(learning_rate=base_learning_rate, momentum=0.9)

cnn512_dropout.compile(
    optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

# create class weight
classes = np.unique(y_train)
class_weights = compute_class_weight("balanced", classes=classes, y=y_train)
class_weight_dict = dict(zip(classes, class_weights))


# create triangular schedule
def triangular_schedule(epoch):
    """Triangular learning rate scheduler."""
    cycle_length = 10  # Define the length of a cycle
    cycle = math.floor(1 + epoch / (2 * cycle_length))
    x = abs(epoch / cycle_length - 2 * cycle + 1)
    lr = base_learning_rate + (max_learning_rate - base_learning_rate) * max(0, (1 - x))
    return lr


# When fitting the model, include the learning rate scheduler callback
lr_scheduler = LearningRateScheduler(triangular_schedule)

# fitting the model
cnn512_dropout.fit(
    x_train,
    y_train,
    epochs=epochs,
    class_weight=class_weight_dict,
    callbacks=[lr_scheduler],
)

# evaluating the model
cnn512_dropout.evaluate(x_test, y_test)

## *CNN299* 

Layer	Operator	Layer Details

Input Layer	Zero Padding layer	Padding (2,2)

Layer 1	2D CONV layer	Kernel number = 32, kernel size = 3

Layer 2	Batch Normalization layer	-

Layer 3	Relu layer	-

Layer 4	Max Pooling layer	Pooling size (2,2)

Layer 5	2D CONV layer	Kernel number = 64, kernel size = 3

Layer 6	Batch Normalization layer	-

Layer 7	Relu layer	-

Layer 8	Max Pooling layer	Pooling size (2,2)

Layer 9	2D CONV layer	Kernel number = 96, kernel size = 3

Layer 10	Batch Normalization layer	-

Layer 11	Relu layer	-

Layer 12	Max Pooling layer	Pooling size (2,2)

Layer 13	2D CONV layer	Kernel number = 96, kernel size = 3

Layer 14	Batch Normalization layer	-

Layer 15	Relu layer	-

Layer 16	Max Pooling layer	Pooling size (2,2)

Layer 17	Flatten layer	-

Layer 18	FC layer	Neurons number = 1000

Layer 19	Batch Normalization layer	-

Layer 20	Relu layer	-

Layer 21	FC layer	Neurons number = 500

Layer 22	Batch Normalization layer	-

Layer 23	Relu layer	-

Layer 24	FC layer	With SoftMax activation


In [10]:
input_shape = (299, 299, 3)  # placeholder
num_classes = 5

CNN299 Without Dropout

In [11]:
cnn299_nodropout = models.Sequential(
    [
        # Input Layer Zero Padding layer Padding (2,2)
        layers.ZeroPadding2D(padding=(2, 2), input_shape=input_shape),
        # Layer 1 2D CONV layer Kernel number = 32, kernel size = 3
        layers.Conv2D(32, kernel_size=(3, 3), padding="same"),
        # Layer 2 Batch Normalization layer
        layers.BatchNormalization(),
        # Layer 3 Relu layer
        layers.Activation("relu"),
        # Layer 4 Max Pooling layer Pooling size (2,2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 5 2D CONV layer Kernel number = 64, kernel size = 3
        layers.Conv2D(64, kernel_size=(3, 3), padding="same"),
        # Layer 6 Batch Normalization layer
        layers.BatchNormalization(),
        # Layer 7 Relu layer
        layers.Activation("relu"),
        # Layer 8 Max Pooling layer Pooling size (2, 2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 9 2D CONV layer Kernel number = 96, kernel size = 3
        layers.Conv2D(96, kernel_size=(3, 3), padding="same"),
        # Layer 10 Batch Normalization layer
        layers.BatchNormalization(),
        # Layer 11 Relu layer
        layers.Activation("relu"),
        # Layer 12 Max Pooling layer Pooling size (2, 2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 13 2D CONV layer Kernel number = 96, kernel size = 3
        layers.Conv2D(96, kernel_size=(3, 3), padding="same"),
        # Layer 14 Batch Normalization layer
        layers.BatchNormalization(),
        # Layer 15 Relu layer
        layers.Activation("relu"),
        # Layer 16 Max Pooling layer Pooling size (2, 2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 17 Flatten Layer -
        layers.Flatten(),
        # Layer 18 FC layer Neurons number = 1000
        layers.Dense(1000),
        # Layer 19 Batch Normalization layer
        layers.BatchNormalization(),
        # Layer 20 Relu layer
        layers.Activation("relu"),
        # Layer 21 FC layer Neurons number = 500
        layers.Dense(500),
        # Layer 22 Batch Normalization layer
        layers.BatchNormalization(),
        # Layer 23 Relu layer
        layers.Activation("relu"),
        # Layer 24 FC layer with Softmax activation for class predictions
        layers.Dense(num_classes, activation="softmax"),
    ]
)

/Users/tinayiluo/miniconda3/lib/python3.11/site-packages/keras/src/layers/reshaping/zero_padding2d.py:72: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [12]:
cnn299_nodropout.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ zero_padding2d_2                │ (None, 303, 303, 3)    │             0 │
│ (ZeroPadding2D)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 303, 303, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_16          │ (None, 303, 303, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_16 (Activation)      │ (None, 303, 303, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 151, 151, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 151, 151, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_17          │ (None, 151, 151, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_17 (Activation)      │ (None, 151, 151, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_13 (MaxPooling2D) │ (None, 75, 75, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 75, 75, 96)     │        55,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_18          │ (None, 75, 75, 96)     │           384 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_18 (Activation)      │ (None, 75, 75, 96)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_14 (MaxPooling2D) │ (None, 37, 37, 96)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_15 (Conv2D)              │ (None, 37, 37, 96)     │        83,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_19          │ (None, 37, 37, 96)     │           384 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_19 (Activation)      │ (None, 37, 37, 96)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_15 (MaxPooling2D) │ (None, 18, 18, 96)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 31104)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 1000)           │    31,105,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_20          │ (None, 1000)           │         4,000 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_20 (Activation)      │ (None, 1000)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 500)            │       500,50

 Total params: 31,772,981 (121.20 MB)

 Trainable params: 31,769,405 (121.19 MB)

 Non-trainable params: 3,576 (13.97 KB)

In [ ]:
# base learning rate for custom CNNs
base_learning_rate = 1e-4
# maximum learning rate for custom CNNs
max_learning_rate = 1e-1

# Create an instance of SGD optimizer with initial learning rate
optimizer = SGD(learning_rate=base_learning_rate, momentum=0.9)

cnn299_nodropout.compile(
    optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

# create class weight
classes = np.unique(y_train)
class_weights = compute_class_weight("balanced", classes=classes, y=y_train)
class_weight_dict = dict(zip(classes, class_weights))


# create triangular schedule
def triangular_schedule(epoch):
    """Triangular learning rate scheduler."""
    cycle_length = 10  # Define the length of a cycle
    cycle = math.floor(1 + epoch / (2 * cycle_length))
    x = abs(epoch / cycle_length - 2 * cycle + 1)
    lr = base_learning_rate + (max_learning_rate - base_learning_rate) * max(0, (1 - x))
    return lr


# When fitting the model, include the learning rate scheduler callback
lr_scheduler = LearningRateScheduler(triangular_schedule)

# fitting the model
cnn299_nodropout.fit(
    x_train,
    y_train,
    epochs=epochs,
    class_weight=class_weight_dict,
    callbacks=[lr_scheduler],
)

# evaluating the model
cnn299_nodropout.evaluate(x_test, y_test)

> CNN299 With Dropout

In [13]:
cnn299_dropout = models.Sequential(
    [
        # Input Layer Zero Padding layer Padding (2,2)
        layers.ZeroPadding2D(padding=(2, 2), input_shape=input_shape),
        # Layer 1 2D CONV layer Kernel number = 32, kernel size = 3
        layers.Conv2D(32, kernel_size=(3, 3), padding="same"),
        # Layer 2 Batch Normalization layer
        layers.BatchNormalization(),
        # Layer 3 Relu layer
        layers.Activation("relu"),
        # Layer 4 Max Pooling layer Pooling size (2,2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 5 2D CONV layer Kernel number = 64, kernel size = 3
        layers.Conv2D(64, kernel_size=(3, 3), padding="same"),
        # Layer 6 Batch Normalization layer
        layers.BatchNormalization(),
        # Layer 7 Relu layer
        layers.Activation("relu"),
        # Layer 8 Max Pooling layer Pooling size (2, 2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 9 2D CONV layer Kernel number = 96, kernel size = 3
        layers.Conv2D(96, kernel_size=(3, 3), padding="same"),
        # Layer 10 Batch Normalization layer
        layers.BatchNormalization(),
        # Layer 11 Relu layer
        layers.Activation("relu"),
        # Layer 12 Max Pooling layer Pooling size (2, 2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 13 2D CONV layer Kernel number = 96, kernel size = 3
        layers.Conv2D(96, kernel_size=(3, 3), padding="same"),
        # Layer 14 Batch Normalization layer
        layers.BatchNormalization(),
        # Layer 15 Relu layer
        layers.Activation("relu"),
        # Layer 16 Max Pooling layer Pooling size (2, 2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 17 Flatten Layer -
        layers.Flatten(),
        # Layer 18 FC layer Neurons number = 1000
        layers.Dense(1000),
        # Layer 19 Batch Normalization layer
        layers.BatchNormalization(),
        # Layer 20 Relu layer
        layers.Activation("relu"),
        layers.Dropout(0.5),
        # Layer 21 FC layer Neurons number = 500
        layers.Dense(500),
        # Layer 22 Batch Normalization layer
        layers.BatchNormalization(),
        # Layer 23 Relu layer
        layers.Activation("relu"),
        layers.Dropout(0.5),
        # Layer 24 FC layer with Softmax activation for class predictions
        layers.Dense(num_classes, activation="softmax"),
    ]
)

/Users/tinayiluo/miniconda3/lib/python3.11/site-packages/keras/src/layers/reshaping/zero_padding2d.py:72: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [14]:
cnn299_dropout.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ zero_padding2d_3                │ (None, 303, 303, 3)    │             0 │
│ (ZeroPadding2D)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_16 (Conv2D)              │ (None, 303, 303, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_22          │ (None, 303, 303, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_22 (Activation)      │ (None, 303, 303, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_16 (MaxPooling2D) │ (None, 151, 151, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_17 (Conv2D)              │ (None, 151, 151, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_23          │ (None, 151, 151, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_23 (Activation)      │ (None, 151, 151, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_17 (MaxPooling2D) │ (None, 75, 75, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_18 (Conv2D)              │ (None, 75, 75, 96)     │        55,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_24          │ (None, 75, 75, 96)     │           384 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_24 (Activation)      │ (None, 75, 75, 96)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_18 (MaxPooling2D) │ (None, 37, 37, 96)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_19 (Conv2D)              │ (None, 37, 37, 96)     │        83,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_25          │ (None, 37, 37, 96)     │           384 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_25 (Activation)      │ (None, 37, 37, 96)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_19 (MaxPooling2D) │ (None, 18, 18, 96)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 31104)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 1000)           │    31,105,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_26          │ (None, 1000)           │         4,000 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_26 (Activation)      │ (None, 1000)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 1000)           │             

 Total params: 31,772,981 (121.20 MB)

 Trainable params: 31,769,405 (121.19 MB)

 Non-trainable params: 3,576 (13.97 KB)

In [ ]:
# base learning rate for custom CNNs
base_learning_rate = 1e-4
# maximum learning rate for custom CNNs
max_learning_rate = 1e-1

# Create an instance of SGD optimizer with initial learning rate
optimizer = SGD(learning_rate=base_learning_rate, momentum=0.9)

cnn299_dropout.compile(
    optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

# create class weight
classes = np.unique(y_train)
class_weights = compute_class_weight("balanced", classes=classes, y=y_train)
class_weight_dict = dict(zip(classes, class_weights))


# create triangular schedule
def triangular_schedule(epoch):
    """Triangular learning rate scheduler."""
    cycle_length = 10  # Define the length of a cycle
    cycle = math.floor(1 + epoch / (2 * cycle_length))
    x = abs(epoch / cycle_length - 2 * cycle + 1)
    lr = base_learning_rate + (max_learning_rate - base_learning_rate) * max(0, (1 - x))
    return lr


# When fitting the model, include the learning rate scheduler callback
lr_scheduler = LearningRateScheduler(triangular_schedule)

# fitting the model
cnn299_dropout.fit(
    x_train,
    y_train,
    epochs=epochs,
    class_weight=class_weight_dict,
    callbacks=[lr_scheduler],
)

# evaluating the model
cnn299_dropout.evaluate(x_test, y_test)